# Week 4: Best Fit Lines and the Theis Solution

Today let's take a look at withdrawals from an industry in the East St. Louis area. These withdrawals include both groundwater and surface water.

We have already discussed that the area has experienced drastic reductions in groundwater withdrawals compared to the 1950's, but how has this changed in the past few decades (1980 onward)?

Furthermore, are there any trends that we can use to extrapolate expected water demands (usage) out to 2050 for this region?

## Import Packages/Libraries

In [0]:
# import packages
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as spo
import scipy.special as sps
import math

%matplotlib inline

## Import Demands for an industry in Madison County

<ul>
    <li>Surface Water (sw)</li>
    <li>Groundwater (gw)</li>
</ul>

In [2]:
gc_df = pd.read_excel('https://github.com/yiquanw2/geol572_4/blob/master/industry.xlsx?raw=true',index_col=0)
print(gc_df)

HTTPError: ignored

## Plot industrial demands

In [0]:
gc_df.plot(style='o')
plt.ylabel('Mgd')

## Define a linear function to create a best fit line

When defining a function, always use the following syntax:

`def functionname(argument1,argument2,.....):`</br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; `write code here`</br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; `return output1, output2,.....`

To call the function, write

`output1, output2 = functionname(argument1,argument2)`

In [0]:
# define a function that defines our line
def linear(time,m,b):
    return m*time+b

# code that returns the parameters m & b
para, cova = spo.curve_fit(linear,gc_df.index,gc_df.gw,p0=[0.0000001,0.0000001])
print(para)
slope = para[0]
yint = para[1]

year = gc_df.index.to_series()
fitline = linear(year,slope,yint)
print(type(fitline))

gc_df.gw.plot(style='o')
fitline.plot()

## Linear trend to 2050?

In [0]:
linear(2050,slope,yint)

### Theis solution

Recall that the Theis solution is:

<font size = 10>$s=\frac{Q}{4\pi T} W(\frac{r^2S}{4Tt})$</font>

where s is drawdown, Q is pumpage, T is transmissivity, r is distance from well, S is storage, and t is time the well has been on. The well function has many names, with the most formal mathematical name being the `exponential integral`. For the mathematicians out there, [here is more context on the exponential integral.](http://mathworld.wolfram.com/ExponentialIntegral.html). For our purposes, we don't have to work with the exponential integral any differently than we do a natural logarithm or sinusoidal function. We just have to know where it is located. It turns out it is in the SciPy library as a [special function](https://docs.scipy.org/doc/scipy/reference/special.html).

Let's do an example problem related to the NE Illinois sandstone. Assume that we want to know how much drawdown a new well might have on a nearby well (500 ft away) after running continuously for 1 day? 1 week? 1 year? Assume the well pumps 133,000 $ft^3/day$, transmissivity is 1200 $ft^2/day$, $S_s$ is 2.6E-7 $\frac{1}{ft}$, and aquifer thickness is 400 $ft$.

In [0]:
Q = -133000 #ft^3/d
T = 1200 #ft^2/d
S_s = 2.6*10**-7 #1/ft
b = 400 #ft
S = b*S_s
r = 5000 #ft

t = pd.Series(range(0,366))
u = r**2*S/(4*T*t)

s = Q/(4*math.pi*T)*sps.exp1(u)

s.plot()
plt.title('Theis Solution')
plt.xlabel('Days')
plt.ylabel('Drawdown (ft)')
plt.show()